<a href="https://colab.research.google.com/github/ElAm1ne/March--Fi-2/blob/master/SuperHedge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To do :    


1.   Max(VT - Payoff) + Minimize V0
2.   V0/S0 & MAX(VT - Payoff)
3.   Max(VT - Payoff)/(V0/S0)
Mettre en func de High Low



---


Installe automatiquement les librairies manquantes à l'execution du programme

***(peut prendre 5 à 10 minutes si aucune des librairies n'est déjà installée)***

---



In [ ]:
import sys
import subprocess
import pkg_resources

required = {'scipy', 'numpy', 'pandas', 'openpyxl', 'numba', 'matplotlib', 'plotly'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import pandas as pd
import numpy as np
import math
import numba as nb
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from sys import argv
import os
from scipy import stats
import plotly.graph_objects as go
from plotly.graph_objs import Candlestick
from numba import jit, types
from numba.typed import List
from numba.extending import get_cython_function_address
from numba import vectorize, njit
import ctypes
from google.colab import drive
from google.colab import files
import plotly.express as px

---

***(Lecture du fichier)***

---

In [ ]:
!wget https://raw.githubusercontent.com/ElAm1ne/March--Fi-2/master/data.xlsx
file = 'data.xlsx'
df = [pd.read_excel(file, sheet_name = sheet) for sheet in ['CAC', 'DAX', 'IBEX35TR', 'IBEX', 'SX5E', 'SX5T', 'SMI']]
for i in range(len(df)):
    df[i] = df[i].drop(0)
prices = df[0]["PX_LAST"].to_numpy(dtype = np.float64)

--2022-12-27 08:22:55--  https://raw.githubusercontent.com/ElAm1ne/March--Fi-2/master/data.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3106063 (3.0M) [application/octet-stream]
Saving to: ‘data.xlsx’

data.xlsx           100%[===================>]   2.96M  --.-KB/s    in 0.01s   

2022-12-27 08:22:55 (234 MB/s) - ‘data.xlsx’ saved [3106063/3106063]



---


Calcul la valeur critique de l'interval de confiance en utilisant la fonction de répartition de la loi student, fonction compilée et executée en Fortran wrapper en C ensuite éxecutée Python.

$F(t) = \frac{\Gamma(\frac{\nu+1}{2})}{\sqrt{\nu \pi} \Gamma(\frac{\nu}{2})} \int_{-\infty}^{t} (1 + \frac{x^2}{\nu})^{-\frac{\nu+1}{2}} \, dx$

La fonction est très complexe à calculer car elle passe par un calcul d'integrale (fonction Gamma d'Euler $\Gamma(z) = \int_0^{\infty} t^{z-1} e^{-t} \, dt$) et le calcul des series (fonction hypergeometrique $F(a, b, c; z) =$ $\sum_{n=0}^{\infty} \frac{(a)_n (b)_n}{(c)_n} \frac{z^n}{n!}$). En bas du notebook existe des implémentations sous Python de ces fonctions qui permettait de réduire le temps d'execution à quelques microsecondes mais au vu de la nécessité de plusieurs itérations (de l'ordre $10^5$), le passage à une fonction codé en Fortran wrappé était nécessaire.

***(Sera utilisée dans le calcul des intervalles de confiance de la moyenne en raison du nombre faible d'échantillion)***

---

In [ ]:
_PTR = ctypes.POINTER
_dble = ctypes.c_double
_ptr_dble = _PTR(_dble)

addr = get_cython_function_address("scipy.special.cython_special", "stdtrit")
functype = ctypes.CFUNCTYPE(_dble, _dble, _dble)
t_ppf = functype(addr)

@njit('float64(float64, float64)', fastmath = True)
def numba_ppf(x,y):
  return(t_ppf(x,y))

---


Création des fonctions Somme, Moyenne, Ecart Type et Médiane maison en raison des temps d'éxecution élevé des librairies Numpy, Math ou Scipy pour ces fonctions.

Le décorateur @njit provient de la librairie Numba, une librairie d'accélération des codes pythons très pratiques et sera utilisée intensivement dans ce projet. **Des réductions des temps d'éxecution de x10 à x100.**

---

In [ ]:
@njit(fastmath = True, cache = True)
def somme(L):
  s: float = 0
  for i in nb.prange(len(L)):
    s+=L[i]
  return s

In [ ]:
@njit(cache = True, fastmath = True)
def mediane(L):
  return np.sort(L)[int(len(L)/2)]

In [ ]:
@njit('float64(float64[:])', cache = True, fastmath = True)
def moy(x):
    l = len(x)
    s = 0
    for i in range(l):
        s+= x[i]
    return s/l

In [ ]:
@njit(cache = True, fastmath = True)
def et(x, m):
    n = len(x)
    s = 0
    for i in nb.prange(n):
        s+= (x[i] - m)**2
    s = s/n
    s = np.sqrt(s)
    return s

---


Définition des fonctions **g(t,x)** et **Teta(t,x)**

---

In [ ]:
@nb.njit(nb.types.float64(nb.types.int64, nb.types.int64, nb.types.float64, nb.types.float64[:], nb.types.float64[:], nb.types.float64),fastmath = True, cache = True)
def g(t, T, K, Ku, Kd, x):
  if t >= T:
    return max(x - K, 0)
  else:
    if Ku[t] == Kd[t]:
      return 0
    lamda = (1 - Kd[t]) / (Ku[t] - Kd[t])
    return g(t+1, T, K, Ku, Kd, Ku[t] * x) * lamda + g(t+1, T, K, Ku, Kd, Kd[t] * x) * (1 - lamda)

In [ ]:
@njit(fastmath = True)
def Teta(prices, ic, st, t, K, T):
  Supp_Ku = np.ones(T, dtype = np.float64)
  Supp_Kd = np.ones(T, dtype = np.float64)
  for i in nb.prange(T):
    input = prices[i:]
    today = input[-1]
    logged_prices = np.log(input)
    n = len(logged_prices)
    m = moy(logged_prices)
    s = et(logged_prices, m)
    mu = m + (s**2)/2
    z = numba_ppf(n, (1+ic)/2)
    err = (s**2/n) + (s**4/(2*(n-1)))
    interval = z * np.sqrt(err)
    Ku = np.exp(mu + interval) / today
    Kd = np.exp(mu - interval)/ today
    Supp_Ku[i] = max(Ku,1)
    Supp_Kd[i] = min(Kd,1)
  qte: float = 0
  V0 : float = 0
  if t == 0:
    V0 = g(int(t), int(T), float(K), Supp_Ku, Supp_Kd, float(st))
    qte = (g(int(t+1), int(T), float(K), Supp_Ku, Supp_Kd, float(Supp_Ku[t]*st)) - g(int(t+1), int(T), float(K), Supp_Ku, Supp_Kd, float(Supp_Kd[t]*st))) / ((Supp_Ku[t] - Supp_Kd[t])*st)
  else:
    qte = (g(int(t+1), int(T), float(K), Supp_Ku, Supp_Kd, float(Supp_Ku[t]*st)) - g(int(t+1), int(T), float(K), Supp_Ku, Supp_Kd, float(Supp_Kd[t]*st))) / ((Supp_Ku[t] - Supp_Kd[t])*st)

  return qte, V0

---


Nous définissons ici notre fonction **BackTestSimulation** qui permet de backtester la stratégie sur toute la période de **2000 à 2019**

Elle sera ensuite utiliser intensivement par notre optimisateur.

---

In [ ]:
@njit(fastmath = True)
def BackTestSimulation(data, ic: float, n_train: int, T: int):
  #qte = []
  #Vt = []
  V0 = np.zeros(len(data) - n_train - T, dtype = np.float64)
  S0 = np.zeros(len(data) - n_train - T, dtype = np.float64)
  Payoff = np.zeros(len(data) - n_train - T, dtype = np.float64)
  VT = np.zeros(len(data) - n_train - T, dtype = np.float64)
  for j in nb.prange(len(data) - n_train - T):
    window = prices[j:j+n_train+T]
    K: float = mediane(window)
    #Qt_window = []
    #Vt_window = []
    for i in nb.prange(n_train, len(window)):
      train = window[:i]
      qt, V0_window = Teta(train, ic, window[i], i - n_train, K, T)
      #Qt_window.append(qt) #Avoir une trajectoire de quantités pour chaque window
      if i==n_train:
        V0[j] = V0_window #Avoir une liste des V0 pour chaque window
        Vt_: float = V0_window + qt*(window[i] - window[i-1])
        S0[j] = window[i]
      else:
        Vt_ = Vt_ + qt*(window[i] - window[i-1])
      #Vt_window.append(Vt_) #Avoir une trajectoire de Vt pour chaque window
      if i==len(window) - 1 :
        payoff: float = max(window[i] - K, 0)
    #Vt.append(Vt_window) #Liste des trajectoires de Vt
    VT[j] = Vt_ #créer une liste des VT
    Payoff[j] = payoff #créer une liste des Payoffs
    #qte.append(Qt_window) #Liste des trajectoires des Quantités t
  return VT, Payoff, V0, S0

Test de la fonction avec un **lookback de 30 jours** et des Calls de **maturité 10 jours** avec un **intervalle de confiance de 99%**

In [ ]:
n_train = 20 #durée du lookback
n_test = 10 #équivalent à la maturité de l'option
ic = 0.3 #confiance en %

In [ ]:
VT, Payoff, V0, S0 = BackTestSimulation(prices, ic, n_train, n_test)

In [ ]:
obj = somme((VT - Payoff)[VT - Payoff >=0])*0.35-45*somme((VT - Payoff)[VT - Payoff <0])-2*somme(V0)
print(obj)
print(somme((VT - Payoff)[VT - Payoff >=0])/obj)
print(somme((VT - Payoff)[VT - Payoff <0])/obj)
print(somme(V0)/obj)

13842.680258025648
34.57580296456789
-0.19863982435537259
10.020161566795263


In [ ]:
0.33/(somme((VT - Payoff)[VT - Payoff >=0])/(somme((VT - Payoff)[VT - Payoff >=0]) + somme((VT - Payoff)[VT - Payoff <0])+somme(V0)))

0.3963064902278344

In [ ]:
0.33/(somme((VT - Payoff)[VT - Payoff <0])/(somme((VT - Payoff)[VT - Payoff >=0]) + somme((VT - Payoff)[VT - Payoff <0])+somme(V0)))

-44.424315954527096

In [ ]:
0.33/(somme(V0)/(somme((VT - Payoff)[VT - Payoff >=0]) + somme((VT - Payoff)[VT - Payoff <0])+somme(V0)))

1.8885254570255572



---


Le graphe ci dessous représente les résultats du **BackTest jour par jour**, vous pouvez zoomer, choisir un intervalle précis de jour à regarder, l'air entre les courbes <font color='red'>**VT**</font> (Valeur du ptf à maturité) et le <font color='blue'>**Payoff**</font> est en <font color='green'>**vert**</font> quand la sureplication est bonne et en <font color='red'>**rouge**</font> dans le cas inverse.



---



In [ ]:
Valeur_portefeuille = VT
Payoff = Payoff

difference = [Valeur_portefeuille[i] - Payoff[i] for i in range(len(Valeur_portefeuille))]

candle_data = []
for i in range(len(Valeur_portefeuille)):
    if difference[i] > 0:
        candle_data.append(dict(x=i, open=Payoff[i], high=Payoff[i], low=Valeur_portefeuille[i], close=Valeur_portefeuille[i]))
    elif difference[i] < 0:
        candle_data.append(dict(x=i, open=Payoff[i], high=Payoff[i], low=Valeur_portefeuille[i], close=Valeur_portefeuille[i]))
    else:
        candle_data.append(dict(x=i, open=Payoff[i], high=Payoff[i], low=Valeur_portefeuille[i], close=Valeur_portefeuille[i]))
fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille)), y=Valeur_portefeuille, name='Valeur portefeuille en T',
                                 line=dict(color='#339933'), marker=dict(color='#339933')),
                      go.Scatter(x=np.arange(0,len(Valeur_portefeuille)), y=Payoff, name='Payoff en T',
                                 line=dict(color='#9933CC'), marker=dict(color='#9933CC')),
                      go.Candlestick(x=np.arange(0,len(Valeur_portefeuille)), open=list(map(lambda x: x['open'], candle_data)), high=list(map(lambda x: x['high'], candle_data)), low=list(map(lambda x: x['low'], candle_data)), close=list(map(lambda x: x['close'], candle_data)), increasing_line_color= '#90EE90', decreasing_line_color= 'red', name = 'Différence VT - Payoff')])
fig.update_layout(title='Valeur portefeuille en T vs Payoff en T',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))



---


Le graphe ci dessous représente la différence <font color='red'>**VT**</font> - <font color='blue'>**Payoff**</font>.



---



In [ ]:
Valeur_portefeuille = VT
Payoff = Payoff
Diff = [Valeur_portefeuille[i] - Payoff[i] for i in range(len(Payoff))]


fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=Diff, name='VT - payoff en T',
                                 line=dict(color='#339933'), marker=dict(color='#339933'))])

fig.update_layout(title='VT - payoff en T',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.update_layout(font=dict(family='Courier New, monospace', size=14))


fig.update_traces(line=dict(color='#00BFFF', width=2))

fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
fig.show()

---


Nous remarquons que la suréplication est bonne dans la majorité des cas, mais ceci n'est pas suffisant, l'un des moyens insidieux d'assurer une bonne sureplication est de simplement augmenter le $V_0 = g(0,S_0)$.

Autrement dit, choisir des $K_u$ et $K_d$ très conservateurs $\Leftrightarrow$ $\text{Supp}_{F_t}(S_{t+1})$ très large $\Rightarrow$ $V_0$ élevé.

Le soucis est que nous sommes dans un marché, si $V_0$ est élevé pratiquement, nous ne trouverons pas d'acheteur.

Ci-dessous les prix des options à l'instant $t = 0$, $V_0$ :

---


In [ ]:
fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=V0, name='Prix V0',
                                 line=dict(color='#339933'), marker=dict(color='#339933'))])

fig.update_layout(title='Prix V0',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.update_layout(font=dict(family='Courier New, monospace', size=14))


fig.update_traces(line=dict(color='#00BFFF', width=2))

fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
fig.show()



---


Un métrique qui permetterait de mesurer si l'option est cher peut être :    


*   Selon le marché, nous pourrions imaginer : $\frac{V_0}{S_0}$
*   Selon l'acheteur de l'option : $\frac{V_0}{\text{Payoff}}$
*   Selon le vendeur de l'option (nous) : $\frac{V_0}{V_T}$.

Nous choisissons dans un premier temps la mesure par rapport au marché, le graphe ci dessous représente le ratio : $\frac{V_0}{S_0}$


---



In [ ]:
Valeur_portefeuille = VT
Payoff =  Payoff
Diff = [Valeur_portefeuille[i] - Payoff[i] for i in range(len(Payoff))]
V0S0 = np.divide(V0, S0)

fig = go.Figure(data=[go.Scatter(x=np.arange(0,len(Valeur_portefeuille),1), y=V0S0, name='V0/S0',
                                 line=dict(color='red'), marker=dict(color='red'))])

fig.update_layout(title='V0/S0',
                  xaxis_title='Temps (jours)',
                  yaxis_title='Value (Euro)',
                  paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False))
fig.update_layout(font=dict(family='Courier New, monospace', size=14))


fig.update_traces(line=dict(color='#00BFFF', width=2))

fig.update_layout(xaxis_title_font_color='#636363',
                  yaxis_title_font_color='#636363',
                  xaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  yaxis_title_font=dict(family='Verdana', size=14, color='#636363'),
                  title_x = 0.5)
fig.show()
print(min(S0))
print(moy(V0S0))

2403.04
0.012332577792978727


In [ ]:
@njit(fastmath = True, cache = True)
def ratio_sureplication(L, P):
  n = len(L)
  pas = 1/n
  s=0
  for i in nb.prange(n):
    if L[i] >= P[i]:
      s+=pas

  return s

In [ ]:
#M train sizes
#L ICs
@njit(fastmath = True)
def Optimizer(prices, L, M, n_test):
  obj_values1 = np.ones(len(L)*len(M), dtype = np.float64)
  obj_values2 = np.ones(len(L)*len(M), dtype = np.float64)
  obj_values3 = np.ones(len(L)*len(M), dtype = np.float64)
  VT_values = np.ones(len(L)*len(M), dtype = np.float64)
  Payoff_values = np.ones(len(L)*len(M), dtype = np.float64)
  V0S0_values = np.ones(len(L)*len(M), dtype = np.float64)
  IC_values = np.ones(len(L)*len(M), dtype = np.float64)
  n_train_values = np.ones(len(L)*len(M), dtype = np.float64)
  for i in nb.prange(len(L)):
    for j in nb.prange(len(M)):
      VT, Payoff, V0, S0 = BackTestSimulation(prices, L[i], M[j], n_test)
      n_train_values[len(M)*i+j] = M[j]
      IC_values[len(M)*i+j] = L[i]
      ratio_sureplique = ratio_sureplication(VT, Payoff)
      sum_V0S0 = somme(np.divide(V0,S0))
      V0S0_values[len(M)*i+j] = sum_V0S0
      sum_VT = somme(VT)
      VT_values[len(M)*i+j] = sum_VT
      sum_PO = np.sum(Payoff)
      Payoff_values[len(M)*i+j] = sum_PO
      obj_values1[len(M)*i+j] = ((sum_VT - sum_PO)/sum_PO)/(sum_V0S0/50)
      obj_values2[len(M)*i+j] = ratio_sureplique
      obj_values3[len(M)*i+j] = somme((VT - Payoff)[VT - Payoff >=0])*0.35-45*somme((VT - Payoff)[VT - Payoff <0])-2*somme(V0)

  return obj_values1, obj_values2, obj_values3, VT_values, Payoff_values, V0S0_values, IC_values, n_train_values

In [ ]:
L = np.arange(0.01,1,0.01, dtype = np.float64)
M = np.arange(n_test+1, 60,1, dtype = np.int64)
M

array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
       28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44,
       45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [ ]:
obj_values1, obj_values2, obj_values3, VT_values, Payoff_values, V0S0_values, IC_values, n_train_values = Optimizer(prices, L, M, n_test)

In [ ]:
saved = np.array([obj_values1, obj_values2, obj_values3, VT_values, Payoff_values, V0S0_values, IC_values, n_train_values])

In [ ]:
saved

In [ ]:
np.save('output_calculation.npy', saved)

In [ ]:
loaded = np.load('output_calculation.npy')

FileNotFoundError: ignored

In [ ]:
loaded

array([[ 2.08216704e+02,  2.34486043e+02,  2.02440042e+02, ...,
         1.40833151e+02,  1.57690288e+02,  1.54102914e+02],
       [ 9.55307263e-01,  9.49438202e-01,  9.49152542e-01, ...,
         9.69924812e-01,  9.62121212e-01,  9.54198473e-01],
       [ 6.67777780e+03,  4.14998466e+03,  4.16957011e+03, ...,
        -3.74472208e+03, -3.06947228e+03, -2.30507628e+03],
       ...,
       [ 9.52690827e-01,  8.59921884e-01,  9.32218364e-01, ...,
         9.87031355e-01,  9.39362436e-01,  9.48299024e-01],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00],
       [ 1.00000000e+00,  1.00000000e+00,  1.00000000e+00, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+00]])

In [ ]:
x = []
y = []
z = []
for i in range(int(len(obj_values1)*0.5),len(obj_values1)):
  x.append(L[i//len(M)])
  y.append(M[i%len(M)])
  z.append(obj_values1[i])
x = np.array(x)
y = np.array(y)
z = np.array(z)
print(x)
print(y)
print(z)

[0.5  0.5  0.5  ... 0.99 0.99 0.99]
[35 36 37 ... 57 58 59]
[154.51305155 165.37553067 157.30095491 ... 140.83315142 157.69028754
 154.10291393]


In [ ]:
from scipy.interpolate import griddata
xi = np.sort(x)
yi = np.sort(y)
X,Y = np.meshgrid(xi,yi)
Z = griddata((x,y),z,(X,Y), method='cubic')


In [ ]:
fig = go.Figure(go.Surface(x=xi,y=yi,z=Z))
fig.show()